# Introduction

In this notebook, we will do a comprehensive analysis of the Android app market by comparing thousands of apps in the Google Play store.

# About the Dataset of Google Play Store Apps & Reviews

**Data Source:** <br>
App and review data was scraped from the Google Play Store by Lavanya Gupta in 2018. Original files listed [here](
https://www.kaggle.com/lava18/google-play-store-apps).

# Import Statements

In [6]:
import pandas as pd
import plotly.express as px

# Notebook Presentation

In [7]:
# Show numeric output in decimal format e.g., 2.15
pd.options.display.float_format = '{:,.2f}'.format

# Read the Dataset

In [8]:
df_apps = pd.read_csv('apps.csv')

# Data Cleaning

In [9]:
df_apps.shape

(10841, 12)

In [10]:
df_apps.columns

Index(['App', 'Category', 'Rating', 'Reviews', 'Size_MBs', 'Installs', 'Type',
       'Price', 'Content_Rating', 'Genres', 'Last_Updated', 'Android_Ver'],
      dtype='object')

In [11]:
df_apps.sample(5)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Android_Ver
10021,"Emoji keyboard - Cute Emoticons, GIF, Stickers",TOOLS,4.40,1107320,25.00,"50,000,000",Free,0,Everyone,Tools,"August 7, 2018",4.1 and up
2849,FL Racing Manager 2018 Pro,SPORTS,4.30,340,15.00,"5,000",Paid,$1.99,Everyone,Sports,"March 17, 2018",3.0 and up
9144,Google Fit - Fitness Tracking,HEALTH_AND_FITNESS,3.90,249855,8.80,"10,000,000",Free,0,Everyone,Health & Fitness,"July 24, 2018",Varies with device
7110,The Holy Rosary,LIFESTYLE,4.70,56197,15.00,"1,000,000",Free,0,Everyone,Lifestyle,"August 6, 2018",1.6 and up
6928,Insight Timer - Free Meditation App,HEALTH_AND_FITNESS,4.60,20161,8.80,"1,000,000",Free,0,Everyone,Health & Fitness,"August 1, 2018",Varies with device


Drop Unused Columns

In [12]:
# del df_apps['Last_Updated']
# del df_apps['Android_Ver']

# Find and Remove NaN values in Ratings

In [13]:
numberOfNaNs = sum(pd.isna(df_apps['Rating']))
print(numberOfNaNs)

1474


In [14]:
df_apps_clean = df_apps.dropna()
df_apps_clean.head()

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Android_Ver
21,KBA-EZ Health Guide,MEDICAL,5.00,4,25.00,1,Free,0,Everyone,Medical,"August 2, 2018",4.0.3 and up
28,Ra Ga Ba,GAME,5.00,2,20.00,1,Paid,$1.49,Everyone,Arcade,"February 8, 2017",2.3 and up
47,Mu.F.O.,GAME,5.00,2,16.00,1,Paid,$0.99,Everyone,Arcade,"March 3, 2017",2.3 and up
82,Brick Breaker BR,GAME,5.00,7,19.00,5,Free,0,Everyone,Arcade,"July 23, 2018",4.1 and up
99,Anatomy & Physiology Vocabulary Exam Review App,MEDICAL,5.00,1,4.60,5,Free,0,Everyone,Medical,"August 2, 2018",4.0 and up


In [15]:
numberOfDublicates = df_apps_clean.pivot_table(columns='App', aggfunc='size')


In [16]:
duplicated_rows = df_apps_clean[df_apps_clean.duplicated()]
duplicated_rows.shape


(474, 12)

In [17]:
duplicated_rows.head()

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Android_Ver
946,420 BZ Budeze Delivery,MEDICAL,5.00,2,11.00,100,Free,0,Mature 17+,Medical,"June 6, 2018",4.1 and up
1133,MouseMingle,DATING,2.70,3,3.90,100,Free,0,Mature 17+,Dating,"July 17, 2018",4.4 and up
1196,"Cardiac diagnosis (heart rate, arrhythmia)",MEDICAL,4.40,8,6.50,100,Paid,$12.99,Everyone,Medical,"July 25, 2018",3.0 and up
1231,Sway Medical,MEDICAL,5.00,3,22.00,100,Free,0,Everyone,Medical,"July 25, 2018",5.0 and up
1247,Chat Kids - Chat Room For Kids,DATING,4.70,6,4.90,100,Free,0,Mature 17+,Dating,"July 24, 2018",4.0.3 and up


In [18]:
df_apps_clean[df_apps_clean.App == 'MouseMingle']

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Android_Ver
1132,MouseMingle,DATING,2.70,3,3.90,100,Free,0,Mature 17+,Dating,"July 17, 2018",4.4 and up
1133,MouseMingle,DATING,2.70,3,3.90,100,Free,0,Mature 17+,Dating,"July 17, 2018",4.4 and up


In [19]:
df_apps_clean = df_apps_clean.drop_duplicates(subset=['App', 'Price', 'Type'])
df_apps_clean[df_apps_clean.App == 'MouseMingle']

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Android_Ver
1132,MouseMingle,DATING,2.70,3,3.90,100,Free,0,Mature 17+,Dating,"July 17, 2018",4.4 and up


# Find Highest Rated Apps

In [20]:
df_apps_clean.sort_values('Rating', ascending=False).head()

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Android_Ver
21,KBA-EZ Health Guide,MEDICAL,5.00,4,25.00,1,Free,0,Everyone,Medical,"August 2, 2018",4.0.3 and up
1790,SUMMER SONIC app,EVENTS,5.00,4,61.00,500,Free,0,Everyone,Events,"July 24, 2018",4.4 and up
1769,Yazdani Cd Center EllahAbad Official App,FAMILY,5.00,8,3.80,500,Free,0,Everyone,Entertainment,"January 12, 2018",4.0 and up
985,DW Security,BUSINESS,5.00,6,15.00,100,Free,0,Everyone,Business,"July 25, 2018",4.1 and up
981,EU Exit poll,LIFESTYLE,5.00,10,9.40,100,Free,0,Everyone,Lifestyle,"July 15, 2016",4.1 and up


# Find 5 Largest Apps in terms of Size (MBs)

In [21]:
df_apps_clean.sort_values('Size_MBs', ascending=False).head()

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Android_Ver
9942,Talking Babsy Baby: Baby Games,LIFESTYLE,4.00,140995,100.00,"10,000,000",Free,0,Everyone,Lifestyle;Pretend Play,"July 16, 2018",4.0 and up
10687,Hungry Shark Evolution,GAME,4.50,6074334,100.00,"100,000,000",Free,0,Teen,Arcade,"July 25, 2018",4.1 and up
9943,Miami crime simulator,GAME,4.00,254518,100.00,"10,000,000",Free,0,Mature 17+,Action,"July 9, 2018",4.0 and up
9944,Gangster Town: Vice District,FAMILY,4.30,65146,100.00,"10,000,000",Free,0,Mature 17+,Simulation,"May 31, 2018",4.0 and up
3144,Vi Trainer,HEALTH_AND_FITNESS,3.60,124,100.00,"5,000",Free,0,Everyone,Health & Fitness,"August 2, 2018",5.0 and up


# Find the 5 App with Most Reviews

In [22]:
df_apps_clean.sort_values('Reviews', ascending=False).head()

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Android_Ver
10805,Facebook,SOCIAL,4.10,78158306,5.30,"1,000,000,000",Free,0,Teen,Social,"August 3, 2018",Varies with device
10785,WhatsApp Messenger,COMMUNICATION,4.40,69119316,3.50,"1,000,000,000",Free,0,Everyone,Communication,"August 3, 2018",Varies with device
10806,Instagram,SOCIAL,4.50,66577313,5.30,"1,000,000,000",Free,0,Teen,Social,"July 31, 2018",Varies with device
10784,Messenger – Text and Video Chat for Free,COMMUNICATION,4.00,56642847,3.50,"1,000,000,000",Free,0,Everyone,Communication,"August 1, 2018",Varies with device
10650,Clash of Clans,GAME,4.60,44891723,98.00,"100,000,000",Free,0,Everyone 10+,Strategy,"July 15, 2018",4.1 and up


# Plotly Pie and Donut Charts - Visualise Categorical Data: Content Ratings

In [23]:
ratings = df_apps_clean.Content_Rating.value_counts()
print(ratings)

Everyone           6619
Teen                912
Mature 17+          357
Everyone 10+        305
Adults only 18+       3
Unrated               1
Name: Content_Rating, dtype: int64


In [24]:
figure = px.pie(labels=ratings.index, values=ratings.values, names=ratings.index)
figure.update_traces(textposition='outside', textinfo='percent+label')
figure.show()

In [25]:
figure = px.pie(labels=ratings.index, values=ratings.values, names=ratings.index, hole=0.6)
figure.update_traces(textposition='inside',textfont_size=15, textinfo='percent')
figure.show()

# Numeric Type Conversion: Examine the Number of Installs

In [26]:
df_apps_clean.Installs=df_apps_clean.Installs.astype(str).str.replace(',', '')
df_apps_clean.Installs=pd.to_numeric(df_apps_clean.Installs)
df_apps_clean[['App', 'Installs']].groupby('Installs').count()

,App
Installs,
1,3
5,9
10,69
50,56
100,303
500,199
1000,697
5000,425
10000,987


# Find the Most Expensive Apps, Filter out the Junk, and Calculate a (ballpark) Sales Revenue Estimate

In [27]:
df_apps_clean.Price=df_apps_clean.Price.astype(str).str.replace('$', '')
df_apps_clean.Price=pd.to_numeric(df_apps_clean.Price)
df_apps_clean.sort_values('Price', ascending=False).head(20)


,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Android_Ver
3946,I'm Rich - Trump Edition,LIFESTYLE,3.60,275,7.30,10000,Paid,400.00,Everyone,Lifestyle,"May 3, 2018",4.1 and up
3221,I am Rich Plus,FAMILY,4.00,856,8.70,10000,Paid,399.99,Everyone,Entertainment,"May 19, 2018",4.4 and up
3554,💎 I'm rich,LIFESTYLE,3.80,718,26.00,10000,Paid,399.99,Everyone,Lifestyle,"March 11, 2018",4.4 and up
2775,I Am Rich Pro,FAMILY,4.40,201,2.70,5000,Paid,399.99,Everyone,Entertainment,"May 30, 2017",1.6 and up
5765,I am rich,LIFESTYLE,3.80,3547,1.80,100000,Paid,399.99,Everyone,Lifestyle,"January 12, 2018",4.0.3 and up
1331,most expensive app (H),FAMILY,4.30,6,1.50,100,Paid,399.99,Everyone,Entertainment,"July 16, 2018",7.0 and up
4606,I Am Rich Premium,FINANCE,4.10,1867,4.70,50000,Paid,399.99,Everyone,Finance,"November 12, 2017",4.0 and up
2461,I AM RICH PRO PLUS,FINANCE,4.00,36,41.00,1000,Paid,399.99,Everyone,Finance,"June 25, 2018",4.1 and up
3114,I am Rich,FINANCE,4.30,180,3.80,5000,Paid,399.99,Everyone,Finance,"March 22, 2018",4.2 and up
1946,I am rich (Most expensive app),FINANCE,4.10,129,2.70,1000,Paid,399.99,Teen,Finance,"December 6, 2017",4.0.3 and up


### The most expensive apps sub $250

In [28]:
df_apps_clean = df_apps_clean[df_apps_clean['Price'] < 250]
df_apps_clean.sort_values('Price', ascending=False).head()

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Android_Ver
2281,Vargo Anesthesia Mega App,MEDICAL,4.60,92,32.00,1000,Paid,79.99,Everyone,Medical,"June 18, 2018",4.0.3 and up
1407,LTC AS Legal,MEDICAL,4.00,6,1.30,100,Paid,39.99,Everyone,Medical,"April 4, 2018",4.1 and up
2629,I am Rich Person,LIFESTYLE,4.20,134,1.80,1000,Paid,37.99,Everyone,Lifestyle,"July 18, 2017",4.0.3 and up
2481,A Manual of Acupuncture,MEDICAL,3.50,214,68.00,1000,Paid,33.99,Everyone,Medical,"October 2, 2017",4.0 and up
4264,Golfshot Plus: Golf GPS,SPORTS,4.10,3387,25.00,50000,Paid,29.99,Everyone,Sports,"July 11, 2018",4.1 and up


### Highest Grossing Paid Apps (ballpark estimate)

In [29]:
df_apps_clean['Revenue_Estimate']=df_apps_clean.Installs.mul(df_apps_clean.Price)
df_apps_clean.sort_values('Revenue_Estimate', ascending=False)[:10]

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Android_Ver,Revenue_Estimate
9220,Minecraft,FAMILY,4.50,2376564,19.00,10000000,Paid,6.99,Everyone 10+,Arcade;Action & Adventure,"July 24, 2018",Varies with device,"69,900,000.00"
8825,Hitman Sniper,GAME,4.60,408292,29.00,10000000,Paid,0.99,Mature 17+,Action,"July 12, 2018",4.1 and up,"9,900,000.00"
7151,Grand Theft Auto: San Andreas,GAME,4.40,348962,26.00,1000000,Paid,6.99,Mature 17+,Action,"March 21, 2015",3.0 and up,"6,990,000.00"
7977,Sleep as Android Unlock,LIFESTYLE,4.50,23966,0.85,1000000,Paid,5.99,Everyone,Lifestyle,"June 27, 2018",4.0 and up,"5,990,000.00"
7477,Facetune - For Free,PHOTOGRAPHY,4.40,49553,48.00,1000000,Paid,5.99,Everyone,Photography,"July 25, 2018",4.1 and up,"5,990,000.00"
6594,DraStic DS Emulator,GAME,4.60,87766,12.00,1000000,Paid,4.99,Everyone,Action,"July 19, 2016",2.3 and up,"4,990,000.00"
6082,Weather Live,WEATHER,4.50,76593,4.75,500000,Paid,5.99,Everyone,Weather,"November 21, 2017",Varies with device,"2,995,000.00"
7044,Tasker,TOOLS,4.60,43045,3.40,1000000,Paid,2.99,Everyone,Tools,"June 25, 2018",Varies with device,"2,990,000.00"
7954,Bloons TD 5,FAMILY,4.60,190086,94.00,1000000,Paid,2.99,Everyone,Strategy,"July 13, 2018",2.3.3 and up,"2,990,000.00"
6746,Card Wars - Adventure Time,FAMILY,4.30,129603,23.00,1000000,Paid,2.99,Everyone 10+,Card;Action & Adventure,"February 11, 2016",2.3.3 and up,"2,990,000.00"


# Plotly Bar Charts & Scatter Plots: Analysing App Categories

In [30]:
df_apps_clean.Category.nunique()

33

### Vertical Bar Chart - Highest Competition (Number of Apps)

In [44]:
df_top_categories = df_apps_clean.Category.value_counts()[:10]
bar_char = px.bar(x=df_top_categories.index, y=df_top_categories.index)
bar_char.show()

### Horizontal Bar Chart - Most Popular Categories (Highest Downloads)

In [45]:
df_installations = df_apps_clean.groupby('Category').agg({"Installs":pd.Series.sum})
df_installations.sort_values('Installs', ascending=True, inplace=True)

In [48]:
h_bar = px.bar(x=df_installations.Installs, y=df_installations.index, orientation='h', title='Most Popular Categories')
h_bar.update_layout(xaxis_title='Number of Installs', yaxis_title='Popular Categories')
h_bar.show()

In [50]:
apps_number=df_apps_clean.groupby('Category').agg({'App':pd.Series.count})
merged_data = pd.merge(apps_number, df_installations, on='Category', how='inner')
print(f'The dimensions of the DataFrame are: {merged_data.shape}')
merged_data.sort_values('Installs', ascending=False)

The dimensions of the DataFrame are: (33, 2)


,App,Installs
Category,,
GAME,910,13858762717
COMMUNICATION,257,11039241530
TOOLS,719,8099724500
PRODUCTIVITY,301,5788070180
SOCIAL,203,5487841475
PHOTOGRAPHY,263,4649143130
FAMILY,1606,4437554490
VIDEO_PLAYERS,148,3916897200
TRAVEL_AND_LOCAL,187,2894859300


In [51]:
scatter = px.scatter(merged_data,
                     x='App',
                     y='Installs',
                     title='Category Concentration',
                     size='App',
                     hover_name=merged_data.index,
                     color='Installs')
scatter.update_layout(xaxis_title='Number of Apps',
                      yaxis_title='Installs',
                      yaxis=dict(type='log'))
scatter.show()

# Extracting Nested Data from a Column

In [52]:
    # Split the strings on the semi-colon and then .stack them.
    stack = df_apps_clean.Genres.str.split(';', expand=True).stack()
    print(f'We now have a single column with shape: {stack.shape}')
    num_genres = stack.value_counts()
    print(f'Number of genres: {len(num_genres)}')

We now have a single column with shape: (8562,)
Number of genres: 53


# Colour Scales in Plotly Charts - Competition in Genres

In [53]:
    bar = px.bar(x = num_genres.index[:15], # index = category name
                 y = num_genres.values[:15], # count
                 title='Top Genres',
                 hover_name=num_genres.index[:15],
                 color=num_genres.values[:15],
                 color_continuous_scale='Agsunset')
     
    bar.update_layout(xaxis_title='Genre',
    yaxis_title='Number of Apps',
    coloraxis_showscale=False)
     
    bar.show()

# Grouped Bar Charts: Free vs. Paid Apps per Category

In [54]:
    df_free_vs_paid = df_apps_clean.groupby(["Category", "Type"], as_index=False).agg({'App': pd.Series.count})
    df_free_vs_paid.head()

,Category,Type,App
0,ART_AND_DESIGN,Free,58
1,ART_AND_DESIGN,Paid,3
2,AUTO_AND_VEHICLES,Free,72
3,AUTO_AND_VEHICLES,Paid,1
4,BEAUTY,Free,42


In [55]:
    g_bar = px.bar(df_free_vs_paid,
                   x='Category',
                   y='App',
                   title='Free vs Paid Apps by Category',
                   color='Type',
                   barmode='group')
     
    g_bar.update_layout(xaxis_title='Category',
                        yaxis_title='Number of Apps',
                        xaxis={'categoryorder':'total descending'},
                        yaxis=dict(type='log'))
     
    g_bar.show()


#  Plotly Box Plots: Lost Downloads for Paid Apps



In [61]:
    box = px.box(df_apps_clean,
                 y='Installs',
                 x='Type',
                 color='Type',
                 notched=True,
                 points='all',
                 title='How Many Downloads are Paid Apps Giving Up?')
     
    box.update_layout(yaxis=dict(type='log'))
     
    box.show()

# Plotly Box Plots: Revenue by App Category

In [60]:
    df_paid_apps = df_apps_clean[df_apps_clean['Type'] == 'Paid']

    box = px.box(df_paid_apps, 
                 x='Category', 
                 y='Revenue_Estimate',
                 title='How Much Can Paid Apps Earn?')
     
    box.update_layout(xaxis_title='Category',
                      yaxis_title='Paid App Ballpark Revenue',
                      xaxis={'categoryorder':'min ascending'},
                      yaxis=dict(type='log'))
     
     
    box.show()

# How Much Can You Charge? Examine Paid App Pricing Strategies by Category

In [59]:
    box = px.box(df_paid_apps,
                 x='Category',
                 y="Price",
                 title='Price per Category')
     
    box.update_layout(xaxis_title='Category',
                      yaxis_title='Paid App Price',
                      xaxis={'categoryorder':'max descending'},
                      yaxis=dict(type='log'))
     
    box.show()